[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Danie\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras.losses import binary_crossentropy

ModuleNotFoundError: No module named 'tensorflow'

In [221]:
data=pd.read_csv('Movie_database_BritishAmerican2000-2021.csv')
plots=data.Plot
test=plots[8]
test=re.sub(r'[^\w\s]', '', test)
test=test.lower()
test

'maya is a quickwitted young woman who comes over the mexican border without papers and makes her way to the la home of her older sister rosa rosa gets maya a job as a janitor a nonunion janitorial service has the contract the foulmouthed supervisor can fire workers on a whim and the serviceworkers union has assigned organizer sam shapiro to bring its justice for janitors campaign to the building sam finds maya a willing listener shes also attracted to him rosa resists she has an ailing husband to consider the workers try for public support management intimidates workers to divide and conquer rosa and maya as well as workers and management may be set to collide'

In [222]:
import re
#Switching out hyphons for spaces
plots=[str(plots[num]).replace('-',' ') for num in range(len(plots))]
# Removing punctuation
plots=[re.sub(r'[^\w\s]','', str(plots[num])) for num in range(len(plots))]
# Lowercasing the words
plots=[str(plots[num]).lower() for num in range(len(plots))]

In [223]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#Removing stop words and tokenising

#Define english stop words
stop_words = set(stopwords.words('english'))

#Define function for removing stop words + tokenizing
def stopntokenize(text):
    word_tokens = word_tokenize(text)
    new_text = []
    for w in word_tokens:
        if w not in stop_words:
            new_text.append(w)
    return(new_text)
    

In [224]:
#Stemming and Lemmatizing
from nltk.stem import PorterStemmer

Ps=PorterStemmer()

#Only seems to work well for verbs lol

def verb_stemmer(tokens):
    wordtypes = nltk.pos_tag(tokens)
    for i in range(len(tokens)):
        if wordtypes[i][1] in ['VB','VBD','VBG','VBN','VBP','VBZ']:
            tokens[i] = Ps.stem(tokens[i])

In [225]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

In [226]:
#Only including important words, (i.e. nouns, verbs, adjectives and adverbs)
#['NN','NNP','JJ','JJR','JJS','RB','RBR','RBS','VB','VBD','VBG','VBN','VBP','VBZ']

def wordremover(tokens):
    wordtypes = nltk.pos_tag(tokens)
    tokens_new=[]
    for i in range(len(tokens)):
        if wordtypes[i][1] in ['NN','NNP','JJ','JJR','JJS','VB','VBD','VBG','VBN','VBP','VBZ']:
            tokens_new.append(tokens[i])
    return(tokens_new)


In [227]:
from nltk.corpus import wordnet
def lemmanstem(tokens):
    new_tokens=[]
    for i in range(len(tokens)):
        if wordnet.morphy(tokens[i])==None:
            new_tokens.append(tokens[i])
        else:
            new_tokens.append(wordnet.morphy(tokens[i]))
    return(new_tokens)   

In [228]:
def tokenizer(text):
    token=stopntokenize(text)
    token=wordremover(token)
    token=lemmanstem(token)
    #token=[lemmatizer.lemmatize(token[num]) for num in range(len(token))]
    #verb_stemmer(token)
    token=list(dict.fromkeys(token))
    return(token)

In [229]:
M=101
plots[M]

'pierce brosnan gives one last mission as james bond 007 starting off in north korea bond is betrayed and captured fourteen months later bond is set free but traded for zao rick yune who was captured by mi6 when back in his world bond sets off to track down zao bond gets caught up in yet another scheme which sends him to millionaire gustav graves toby stephens another mi6 agent known as miranda frost rosamund pike is also posing as a friend of graves bond is invited to a presentation held by graves about a satellite found in space which can project a huge laser beam bond must stop this madman with a fellow american agent known as jinx johnson halle berry while bond tries to stop graves and zao will he finally reveal who betrayed him'

In [230]:
tokenizer(plots[M])

['pierce',
 'brosnan',
 'give',
 'last',
 'mission',
 'bond',
 'starting',
 'north',
 'korea',
 'betray',
 'capture',
 'fourteen',
 'set',
 'trade',
 'zao',
 'rick',
 'yune',
 'world',
 'track',
 'get',
 'catch',
 'scheme',
 'send',
 'millionaire',
 'gustav',
 'toby',
 'stephen',
 'mi6',
 'agent',
 'know',
 'miranda',
 'frost',
 'rosamund',
 'pike',
 'posing',
 'friend',
 'invite',
 'presentation',
 'hold',
 'found',
 'space',
 'project',
 'huge',
 'laser',
 'beam',
 'stop',
 'madman',
 'fellow',
 'american',
 'jinx',
 'johnson',
 'halle',
 'berry',
 'reveal']

In [129]:
x=[]
for i in range(4000):
    x.append(len(tokenizer(plots[i]))/len(plots[i]))

sum(x)/4000

0.07894935929105847

In [132]:

len(tokenizer(plots[5]))/len(plots[5])

0.09261576971214018

In [54]:
token_plots=[]
for i in range(len(plots)):
    token_plots.append(tokenizer(plots[i]))

In [169]:
import gensim

dictionary = gensim.corpora.Dictionary(token_plots)

count = 0
for k,v  in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break
        
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

0 aberdeen
1 agre
2 airline
3 alcoholic
4 along
5 ban
6 beg
7 call
8 car
9 clive
10 coke


In [167]:
run=wordnet.synset('run.v.01')
sprint=wordnet.synset('sprint.v.01')
sprint.path_similarity(run)

0.5

In [220]:
test=tokenizer(plots[M])
for i in range(len(test)):
    for j in range(len(test)):
        if wordnet.synsets(test[i])!=[] and wordnet.synsets(test[j])!=[]:
            
            word1=wordnet.synsets(test[i])[0]
            word2=wordnet.synsets(test[j])[0]
            similarity=word1.path_similarity(word2)
            if similarity>0.15 and similarity !=1:
                print(test[i],test[j])
                test[i]='margaret'
new_test=[]
for i in range(len(test)):
    if test[i]!='margaret':
        new_test.append(test[i])
new_test

give space
starting capture
north track
capture presentation
set space
send huge
millionaire stephen
stephen agent
agent friend
friend madman
presentation hold
hold project
space huge
huge reveal
madman fellow
fellow american
american jinx
jinx johnson


['pierce',
 'brosnan',
 'last',
 'mission',
 'bond',
 'korea',
 'betray',
 'fourteen',
 'trade',
 'zao',
 'rick',
 'yune',
 'world',
 'track',
 'get',
 'catch',
 'scheme',
 'gustav',
 'toby',
 'mi6',
 'know',
 'miranda',
 'frost',
 'rosamund',
 'pike',
 'posing',
 'invite',
 'found',
 'project',
 'laser',
 'beam',
 'stop',
 'johnson',
 'halle',
 'berry',
 'reveal']

In [151]:
dog=wordnet.synset('dog.n.01')
dog.lemma_names()
wordnet.synset('.v.01').lemma_names()

['attack', 'assail']

In [201]:

wordnet.synsets(test[5])

[]